<a href="https://colab.research.google.com/github/thingumajig/mpr-prototype/blob/master/embedding_tsne_and_transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !pip uninstall tf-sentencepiece
# !pip uninstall tensorflow

# Preprocess data

In [0]:
!pip install numpy==1.15.0
!pip install sentencepiece==0.1.83
!pip install tf-sentencepiece==0.1.83
!pip install tensorflow==1.14.0
!pip install tensorflow_hub


     |████████████████████████████████| 13.9MB 2.7MB/s 
ERROR: tensorflow 1.15.0 has requirement numpy<2.0,>=1.16.0, but you'll have numpy 1.15.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.16.5
    Uninstalling numpy-1.16.5:
      Successfully uninstalled numpy-1.16.5


     |████████████████████████████████| 1.0MB 2.8MB/s 
     |████████████████████████████████| 2.7MB 2.9MB/s 
     |████████████████████████████████| 109.2MB 307kB/s 
     |████████████████████████████████| 3.2MB 26.9MB/s 
     |████████████████████████████████| 491kB 40.7MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [0]:
%%time
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import tf_sentencepiece
# Graph set up.
g = tf.Graph()
with g.as_default():
  text_input = tf.placeholder(dtype=tf.string, shape=[None])
  embed = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/1")
  embedded_text = embed(text_input)
  init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


CPU times: user 19.6 s, sys: 397 ms, total: 20 s
Wall time: 20.1 s


## Preprocess data

In [0]:
%%time

import pandas as pd

def load_data():
    df = pd.read_hdf('ideas.h5', key='ideas')
    return df

df = load_data()

col2embed=[
    'Название',
    'Описание ситуации',
    'Проблема',
    'Решение',
    'Ожидаемый эффект'
]
new_columns = [na+'_emb' for na in col2embed]
restricted_columns = set(new_columns)
restricted_columns = restricted_columns.difference(set(['Описание ситуации_emb','Решение_emb']))




def calculate(df: pd.DataFrame):
  # Initialize session.
  session = tf.Session(graph=g)
  session.run(init_op)
  global col2embed, new_columns
  
  def calc_embedding(row):
    l=[str(row[na]) for na in col2embed]
    result = session.run(embedded_text, feed_dict={text_input: l})
    return list(result)

  df[new_columns] = df.apply(lambda row: calc_embedding(row), axis=1, result_type='expand').apply(pd.Series)
  return df 

print("Calculate embeddings")
emb_df = calculate(df)




Calculate embeddings
Create all_emb & short_emb


ValueError: ignored

In [0]:
def concat(row):
  global new_columns
  all_emb=[]
  short_emb = []
  for n in new_columns:
    val = list(row[n])
    all_emb += val
    if n in restricted_columns:
      short_emb += val 
  return [all_emb, short_emb]
    
print("Create all_emb & short_emb")
emb_df[['all_emb','short_emb']] = emb_df.apply(lambda row: concat(row), axis=1, result_type='expand').apply(pd.Series)


print("Save embeddings")
emb_df.to_hdf('emb.h5', key='emb')


Create all_emb & short_emb
Save embeddings


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['Название', 'Предприятие', 'Автор инициативы', 'Категория', 'Направление/Функция', 'Производство/Подразделение', 'Комплекс/Отдел', 'Установка', 'Описание ситуации', 'Проблема', 'Решение', 'Ожидаемый эффект', 'Название_emb', 'Описание ситуации_emb', 'Проблема_emb', 'Решение_emb', 'Ожидаемый эффект_emb', 'all_emb', 'short_emb']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [0]:
print("Show embeddings")
from IPython import display
# display(emb_df.head())
emb_df.head()

Show embeddings


,Название,Дата создания,Предприятие,Автор инициативы,Категория,Направление/Функция,Производство/Подразделение,Комплекс/Отдел,Установка,Описание ситуации,Проблема,Решение,Ожидаемый эффект,Название_emb,Описание ситуации_emb,Проблема_emb,Решение_emb,Ожидаемый эффект_emb,all_emb,short_emb
0,Монтаж кабеля обогрева подкрановых рельс мосто...,2018-04-03 11:47:19,ОНПЗ,Андрей Максименко,Прочее,Надежность оборудования,Производство первичной переработки нефти,Комплекс по получению битумов и кокса,Установка замедленного коксования 21-10/3М,1. В зонах с продолжительными зимними периодам...,Обледенение рельсов Температурной удлинение рельс,Применить обогрев рельса греющим кабелем плоск...,"Повышение безопасности эксплуатации ПС, повыше...","[-0.062170576, -0.0609957, 0.046895217, -0.001...","[0.08872238, -0.07770128, 0.038869042, 0.03657...","[0.03992115, -0.057600852, -8.2133825e-05, -0....","[-0.04697304, -0.08458447, 0.056559913, -0.006...","[0.04018711, -0.09316864, 0.051593237, -0.0244...","[-0.062170576, -0.0609957, 0.046895217, -0.001...","[-0.062170576, -0.0609957, 0.046895217, -0.001..."
1,Изменение схемы отключения насоса Н-3 ПСГ,2018-04-17 07:59:32,ОНПЗ,Олег Барышев,"ПБ, ОТ, ООС, ГЗ",Промышленная безопасность,Товарное производство,Товарно-сырьевая база - 1 (ТСБ - 1),-,На участке ПСГ для налива ПТ в автомобильные ц...,После выполнения работ по замене насосного агр...,Изменить эл. схему управления насосом Н-3 и Н-...,"Повышение производственной безопасности, преду...","[-0.0913934, 0.03010722, -0.021338332, -0.0190...","[0.09505776, 0.023044746, 0.057588566, 0.00992...","[0.04484966, -0.0153370425, 0.06273806, -0.001...","[-0.07141615, -0.004467138, 0.030811895, 0.036...","[0.04503489, 0.039481945, -0.0044103293, 0.041...","[-0.0913934, 0.03010722, -0.021338332, -0.0190...","[-0.0913934, 0.03010722, -0.021338332, -0.0190..."
2,Устройство азотной подушки на шаровых резервуа...,2018-04-17 08:38:37,ОНПЗ,Олег Барышев,Прочее,Повышение производственной эффективности,Товарное производство,Товарно-сырьевая база - 1 (ТСБ - 1),"Автоматическая станция смешения бензинов АССБ,...",МТБЭ получаемый на комплексе КТ-1/1 перед отгр...,Шаровые резервуары п.т. 3146 не оборудованы аз...,Оборудовать шаровые резервуары п.т. 3146 систе...,"Снижение безвозвратных потерь, улучшение эколо...","[-0.03108045, 0.047116138, -0.016765647, -0.01...","[0.028300527, -0.061903544, 0.028284496, -0.00...","[0.032620523, -0.012224519, -0.011042369, -0.0...","[-0.025792079, 0.045215957, -0.0069836243, -0....","[0.041609526, -0.018161789, -0.0045285616, -0....","[-0.03108045, 0.047116138, -0.016765647, -0.01...","[-0.03108045, 0.047116138, -0.016765647, -0.01..."
3,Л-24-6. Направление гидроочищенного компонента...,2018-04-26 08:52:32,ОНПЗ,Ольга Губина,Прочее,Повышение производственной эффективности,-,-,-,"В летний период, ввиду низкой потребности рынк...",1. Необходимость изменения режима работы Л-24-...,Организовать работу обоих потоков Л-24-6 на ва...,Эффект достигается за счет увеличения объема п...,"[-0.05698898, -0.015411004, 0.05386731, 0.0075...","[0.009528116, -0.0002822983, -0.04869192, 0.03...","[-0.05343011, -0.034242697, 0.03154913, 0.0182...","[-0.037766412, 0.02097129, 0.015651047, 0.0220...","[-0.060153995, -0.05986381, -0.019658998, 0.02...","[-0.05698898, -0.015411004, 0.05386731, 0.0075...","[-0.05698898, -0.015411004, 0.05386731, 0.0075..."
4,Вовлечения в сырье 43-103 масляных остатков с ...,2018-04-26 09:02:26,ОНПЗ,Ольга Губина,Прочее,Повышение производственной эффективности,Производство глубокой переработки нефти,Комплекс переработки вторичного сырья,Установка каталитического крекинг 43-103,В настоящий момент достаточно часто возникает ...,1. Дефицит сырья для максимальной загрузки обо...,Возможность вовлечения в сырье 43-103 масляных...,Эффект достигается за счет двух возможностей: ...,"[0.07892673, 0.015526595, -0.005868924, -0.038...","[0.09375272, -0.012271765, 0.0012588714, 0.046...","[0.0822131, 0.019237269, 0.0349393, 0.04293571...","[0.0759918

In [0]:
!pip install sklearn-pandas

In [0]:
# df = pd.DataFrame({'name': [1, 2, 3, 4],
#                    'emb':[[1,2],[1,2],[2,3],[2,4]]})

# df['emb'].tolist()
# emb_df['all_emb'].tolist()

# tsne_data = pd.DataFrame([[1,2,3],[2,3,4],[3,4,5],[4,5,6]], columns=['x', 'y', 'z'])
# df = pd.concat([df, tsne_data], axis=1)
# df

from sklearn.manifold import TSNE

X_embedded = TSNE(n_components=3, init='random', 
                  random_state=0, perplexity=50).fit_transform(emb_df['all_emb'].tolist())

tsne_data = pd.DataFrame(X_embedded, columns=['all_x', 'all_y', 'all_z'])
emb_df = pd.concat([emb_df, tsne_data], axis = 1)
print("Save embeddings")
emb_df.to_hdf('emb.h5', key='emb')



Save embeddings


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['Название', 'Предприятие', 'Автор инициативы', 'Категория', 'Направление/Функция', 'Производство/Подразделение', 'Комплекс/Отдел', 'Установка', 'Описание ситуации', 'Проблема', 'Решение', 'Ожидаемый эффект', 'Название_emb', 'Описание ситуации_emb', 'Проблема_emb', 'Решение_emb', 'Ожидаемый эффект_emb', 'all_emb', 'short_emb']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [0]:
from sklearn.manifold import TSNE

X_embedded = TSNE(n_components=3, init='random', 
                  random_state=0, perplexity=100).fit_transform(emb_df['short_emb'].tolist())

tsne_data = pd.DataFrame(X_embedded, columns=['short_x', 'short_y', 'short_z'])
emb_df = pd.concat([emb_df, tsne_data], axis = 1)
print("Save embeddings")
emb_df.to_hdf('emb.h5', key='emb')

Save embeddings


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2377: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['Название', 'Предприятие', 'Автор инициативы', 'Категория', 'Направление/Функция', 'Производство/Подразделение', 'Комплекс/Отдел', 'Установка', 'Описание ситуации', 'Проблема', 'Решение', 'Ожидаемый эффект', 'Название_emb', 'Описание ситуации_emb', 'Проблема_emb', 'Решение_emb', 'Ожидаемый эффект_emb', 'all_emb', 'short_emb']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [0]:
emb_df.head()

,Название,Дата создания,Предприятие,Автор инициативы,Категория,Направление/Функция,Производство/Подразделение,Комплекс/Отдел,Установка,Описание ситуации,Проблема,Решение,Ожидаемый эффект,Название_emb,Описание ситуации_emb,Проблема_emb,Решение_emb,Ожидаемый эффект_emb,all_emb,short_emb,all_x,all_y,all_z,short_x,short_y,short_z
0,Монтаж кабеля обогрева подкрановых рельс мосто...,2018-04-03 11:47:19,ОНПЗ,Андрей Максименко,Прочее,Надежность оборудования,Производство первичной переработки нефти,Комплекс по получению битумов и кокса,Установка замедленного коксования 21-10/3М,1. В зонах с продолжительными зимними периодам...,Обледенение рельсов Температурной удлинение рельс,Применить обогрев рельса греющим кабелем плоск...,"Повышение безопасности эксплуатации ПС, повыше...","[-0.062170576, -0.0609957, 0.046895217, -0.001...","[0.08872238, -0.07770128, 0.038869042, 0.03657...","[0.03992115, -0.057600852, -8.2133825e-05, -0....","[-0.04697304, -0.08458447, 0.056559913, -0.006...","[0.04018711, -0.09316864, 0.051593237, -0.0244...","[-0.062170576, -0.0609957, 0.046895217, -0.001...","[-0.062170576, -0.0609957, 0.046895217, -0.001...",-15.317044,-41.880203,-0.449970,-8.136565,18.628607,-22.616056
1,Изменение схемы отключения насоса Н-3 ПСГ,2018-04-17 07:59:32,ОНПЗ,Олег Барышев,"ПБ, ОТ, ООС, ГЗ",Промышленная безопасность,Товарное производство,Товарно-сырьевая база - 1 (ТСБ - 1),-,На участке ПСГ для налива ПТ в автомобильные ц...,После выполнения работ по замене насосного агр...,Изменить эл. схему управления насосом Н-3 и Н-...,"Повышение производственной безопасности, преду...","[-0.0913934, 0.03010722, -0.021338332, -0.0190...","[0.09505776, 0.023044746, 0.057588566, 0.00992...","[0.04484966, -0.0153370425, 0.06273806, -0.001...","[-0.07141615, -0.004467138, 0.030811895, 0.036...","[0.04503489, 0.039481945, -0.0044103293, 0.041...","[-0.0913934, 0.03010722, -0.021338332, -0.0190...","[-0.0913934, 0.03010722, -0.021338332, -0.0190...",21.009214,1.525687,-13.756093,11.682576,-9.752286,-32.237122
2,Устройство азотной подушки на шаровых резервуа...,2018-04-17 08:38:37,ОНПЗ,Олег Барышев,Прочее,Повышение производственной эффективности,Товарное производство,Товарно-сырьевая база - 1 (ТСБ - 1),"Автоматическая станция смешения бензинов АССБ,...",МТБЭ получаемый на комплексе КТ-1/1 перед отгр...,Шаровые резервуары п.т. 3146 не оборудованы аз...,Оборудовать шаровые резервуары п.т. 3146 систе...,"Снижение безвозвратных потерь, улучшение эколо...","[-0.03108045, 0.047116138, -0.016765647, -0.01...","[0.028300527, -0.061903544, 0.028284496, -0.00...","[0.032620523, -0.012224519, -0.011042369, -0.0...","[-0.025792079, 0.045215957, -0.0069836243, -0....","[0.041609526, -0.018161789, -0.0045285616, -0....","[-0.03108045, 0.047116138, -0.016765647, -0.01...","[-0.03108045, 0.047116138, -0.016765647, -0.01...",21.206608,-0.929410,13.553578,-7.494962,22.263939,11.601188
3,Л-24-6. Направление гидроочищенного компонента...,2018-04-26 08:52:32,ОНПЗ,Ольга Губина,Прочее,Повышение производственной эффективности,-,-,-,"В летний период, ввиду низкой потребности рынк...",1. Необходимость изменения режима работы Л-24-...,Организовать работу обоих потоков Л-24-6 на ва...,Эффект достигается за счет увеличения объема п...,"[-0.05698898, -0.015411004, 0.05386731, 0.0075...","[0.009528116, -0.0002822983, -0.04869192, 0.03...","[-0.05343011, -0.034242697, 0.03154913, 0.0182...","[-0.037766412, 0.02097129, 0.015651047, 0.0220...","[-0.060153995, -0.05986381, -0.019658998, 0.02...","[-0.05698898, -0.015411004, 0.05386731, 0.0075...","[-0.05698898, -0.015411004, 0.05386731, 0.0075...",30.574478,-24.617987,20.526367,1.147984,33.979450,20.862942
4,Вовлечения в сырье 43-103 масляных остатков с ...,2018-04-26 09:02:26,ОНПЗ,Ольга Губина,Прочее,Повышение производственной эффективности,Производство глубокой переработки нефти,Комплекс переработки вторичного сырья,Установка каталитического крекинг 43-103,В настоящий момент достаточно часто возникает ...,1. Дефицит сырья для максимальн

# Visualization

In [0]:
%%time

# Some texts of different lengths.
russian_sentences = ['Повышение уровня промышленной безопасности и исключение риска травмирования рабочего персонала при ремонтных работах.',
                     'Увеличение пространства рабочего места, качества работы , уменьшение времени при выполнении работы, уменьшение риска травматизма при выполнении работ',
                     'Уменьшение риска получения травмы при выполнении работ, уменьшение времени выполнения операции.',
                     'Модернизация ручной тали в узле сбора высокосернистого бензина (УСВНБ)',
                     'Модернизация рабочего места при постановки и снятии заглушек на пк1\1-3']



# Initialize session.
session = tf.Session(graph=g)
session.run(init_op)

# Compute embeddings.
ru_result = session.run(embedded_text, feed_dict={text_input: russian_sentences})

%time
# Compute similarity matrix. Higher score indicates greater similarity.
similarity_matrix_ru = np.inner(ru_result, ru_result)
print(similarity_matrix_ru)


CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 5.96 µs
[[ 1.          0.5129378   0.49835664  0.06688008  0.17615391]
 [ 0.5129378   0.9999999   0.7497405   0.04539309  0.25847852]
 [ 0.49835664  0.7497405   0.9999998  -0.02138579  0.08450961]
 [ 0.06688008  0.04539309 -0.02138579  0.9999999   0.21910405]
 [ 0.17615391  0.25847852  0.08450961  0.21910405  1.        ]]
[[0.97506666 0.5416357  0.36816865]
 [0.42757022 0.8418217  0.34224993]
 [0.2927456  0.3468555  0.8845725 ]]
CPU times: user 8.87 s, sys: 1.11 s, total: 9.98 s
Wall time: 9.24 s


In [0]:
!pip install altair
!pip install pandas

!pip install sklearn


import altair as alt
import pandas as pd

In [0]:
alt.renderers.enable('colab')

print(ru_result.shape)
print(ru_result[0].shape)
data = pd.DataFrame({'y0': list(ru_result[0]),
                     'y1': list(ru_result[1]),
                     'y2': list(ru_result[2]),
                     'y3': list(ru_result[3]),
                     'y4': list(ru_result[4]),
                     'x': [i for i in range(0, len(ru_result[0]))]})
step = 40
overlap = 1
alt.Chart(data).transform_fold(
    ['y0', 'y1', 'y2', 'y3', 'y4'],
    as_=['embedding', 'lv']
  ).mark_area(
    interpolate='monotone',
    fillOpacity=0.8,
    stroke='lightgray',
    strokeWidth=0.2
  ).encode(
    # x='x',
    # y='lv:Q',
    # alt.Color('embedding:N'),
    alt.X('x:Q', title='',scale=alt.Scale(domain=[0,511])), 
    alt.Y(
        'lv:Q',
        title="",
        # scale=alt.Scale(rangeStep=10),
        scale=alt.Scale(range=[step, -step * overlap]),
        axis=None
    ),
    alt.Fill(
        'embedding:N',
        legend=None,
        scale=alt.Scale(scheme='redyellowblue')
    ),
    row=alt.Row(
         'embedding:N',
         title=None,
        #  header=alt.Header(labelAngle=0, labelAlign='right', format='%B')
     )
 ).properties(
     bounds='flush', title='Embeddings', height=step-10, width=800  
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
).configure_title(
    anchor='end'
)

(5, 512)
(512,)


alt.Chart(...)

In [0]:
!pip install plotly

In [0]:
import numpy as np
from sklearn.manifold import TSNE

X_embedded = TSNE(n_components=3, init='random',
                         random_state=0, perplexity=150).fit_transform(ru_result)
X_embedded.shape

# tsne_data = pd.DataFrame({'emb': ['y0','y1','y2','y3','y4',]
#                      'coords': X_embedded
#                      })
embedding_names = ['y0','y1','y2','y3','y4']
tsne_data = pd.DataFrame(X_embedded, columns=['x','y','z'])
# tsne_data.index.name = 'embedding'
tsne_data['embedding'] = embedding_names
print(tsne_data)
print(tsne_data.index.name)
# tsne_data.pivot(columns='bar', values='baz')
# t = tsne_data.unstack(level=1).reset_index()
# print(t)


import plotly.express as px

fig = px.scatter_3d(tsne_data,x='x', y='y',z='z',color='embedding')
fig.show()

            x           y           z embedding
0  -74.424400 -335.433411   29.040329        y0
1  142.339951  244.818268 -260.685028        y1
2  203.134552    3.815986   68.993423        y2
3 -173.915787  -14.023599  326.170929        y3
4 -279.673035  -56.267796 -232.865051        y4
None
